# Updating columns from geometry.

Most desktop GIS software has a calculate geometry operation that will allow you to create a new field or update an existing field with information from the geometry.  We've seen some examples of this in the section on spatial functions for measurement.

# Point geometries

A very common example is creating columns to hold the coordinates of point data, such as a latitude and longitude column.  This is also quite easy in GeoPandas

In [ ]:
%matplotlib inline
import geopandas as gpd

raptor = gpd.read_file("data/Raptor_Nests.shp")
raptor.rename(inplace=True, columns={"postgis_fi":"gid", "lat_y_dd":"latitude", "long_x_dd":"longitude"})
raptor

First lets recalculate the existing latitude and longitude columns. Notice that the values in the latitude and longitude columns do not correspond to the values in the wkt expression of the geometry in the geometry column.  This is a result of a number of historical artifacts about this data that are not terribly important right now. Whats important is that we can update them quite easily.

Remember that the geometry column contains Shapely geometry objects and GeoPandas implements most shapely methods and properties for those objects. These properties include the x and y property for points so we can simply assign those attributes to the GeoPandas column as follows.

In [ ]:
raptor["latitude"] = raptor["geometry"].y
raptor["longitude"] = raptor["geometry"].x
raptor

This updated existing fields to refelect the current geometry but we could just as easily have created new fields, or fields to hold coordinates in a different CRS

In [ ]:
raptor['northing_26913'] = raptor["geometry"].to_crs(epsg=26913).y
raptor['easting_26913'] = raptor["geometry"].to_crs(epsg=26913).x
raptor

# Line geometries

Other possibilities exist for line geometries such as the coordinates of the start, end, or mid point.  

In [ ]:
linear = gpd.read_file("data/Linear_Projects.shp")
linear

In these cases, where geopandas doesn't directly implement a method it is possible to work with the shapely geometries in a custom function that is applied to the geometries property. 

The coords property of a Shapely linestring object returns the points that make up the line. If you want to know how many points are in the linestring you can get the length of that list. 

This can get complicated if you have MultiLineString geometries, in which case the number of vertices in each LineString must be summed up. When a column contains both simple and multi geometries you may need to handle each case seperate as below:

In [ ]:
def n_points(geom):
    if geom.type == 'LineString':
        return len(geom.coords)
    elif geom.type == 'MultiLineString':
        n_points=0
        for part in geom.geoms:
            n_points += len(part.coords)
        return n_points

linear['n_points'] = linear['geometry'].apply(n_points)
linear.sort_values("n_points")

Similarly you can use standard python methods to get the first or last point in a LineString and work with the geometries that way.

In [ ]:
def end_pt(geom):
    if geom.type == 'LineString':
        return geom.coords[-1]

linear['end'] = linear['geometry'].apply(end_pt)
linear

Note that these fields now contain tuples, not points but they could easily be converted to Shapely points for a geodataseries or unpacked into x and y columns for each point.

# Linear referencing with Shapely

Shapely has tools for linear referencing and interpolation along a line by distance or proportion.  More info can be found in the Shapely documentation but a simple example to find the midpoint of a line is below.

In [ ]:
def mid_point(geom):
    return geom.interpolate(0.5, normalized=True)

linear['mid_pt2'] = linear['geometry'].apply(mid_point)
linear

# Polygon geometries

GeoPandas implements a centroid and representative_point method as we have discussed previously.  More complex geometrical calculations can be achieved by applying custom functions to shapely objects as we did with linear geometries.